# DDR Final project
## Doris Pu 
## Krittika
## Sumashree Javaji

We are going to get the data from San Francisco restaurant.

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import mysql.connector
import warnings

driver = webdriver.Chrome(service=Service('C:/Users/yuuud/Desktop/chromedriver-win64/chromedriver.exe'))
time.sleep(5)

driver.implicitly_wait(60)
driver.set_script_timeout(120)
driver.set_page_load_timeout(60)

driver.get('https://www.yelp.com/')
time.sleep(5)

restaurant = driver.find_element(By.LINK_TEXT, 'Restaurants')
restaurant.click()
time.sleep(5)

search_bar = driver.find_element(By.ID, 'search_location')

search_bar.send_keys(Keys.CONTROL + 'a')
search_bar.send_keys(Keys.BACKSPACE)
time.sleep(5)
# sent location information
search_bar.send_keys('San Francisco, CA')
time.sleep(2)

search_button = driver.find_element(By.XPATH, '//button[@aria-label="Search"]')
search_button.click()

In [2]:
base_url = driver.current_url

urls = []

for i in range(10):
    start_param = i * 10                                        # Second page url added'&start=10', third page url added added'&start=20' etc.
    new_url = f"{base_url}&start={start_param}"
    
    driver.get(new_url)
    time.sleep(3)
    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    results = soup.find_all("a", class_="css-19v1rkv")


    for a in results:
        href = a.get('href')
        if "ad_business_id" not in href and "yelp" not in href: # exclude the sponsored restaurants
            full_url = 'https://www.yelp.com' + href            # make it a full url
            print(full_url)
            urls.append(full_url)


https://www.yelp.com/biz/bottega-san-francisco-2?osq=Restaurants
https://www.yelp.com/biz/the-snug-san-francisco?osq=Restaurants
https://www.yelp.com/biz/elenas-mexican-restaurant-san-francisco?osq=Restaurants
https://www.yelp.com/biz/kothai-republic-san-francisco?osq=Restaurants
https://www.yelp.com/biz/blind-butcher-san-francisco-2?osq=Restaurants
https://www.yelp.com/biz/pearl-san-francisco-8?osq=Restaurants
https://www.yelp.com/biz/memento-sf-san-francisco-3?osq=Restaurants
https://www.yelp.com/biz/fog-harbor-fish-house-san-francisco-2?osq=Restaurants
https://www.yelp.com/biz/horsefeather-san-francisco-2?osq=Restaurants
https://www.yelp.com/biz/nopa-san-francisco?osq=Restaurants
https://www.yelp.com/biz/dalida-san-francisco-2?osq=Restaurants
https://www.yelp.com/biz/otra-san-francisco-3?osq=Restaurants
https://www.yelp.com/biz/la-sarrasine-san-francisco-2?osq=Restaurants
https://www.yelp.com/biz/santeria-san-francisco?osq=Restaurants
https://www.yelp.com/biz/movida-san-francisco?os

In [3]:
folder_name = 'restaurants_list'                #Create a folder
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

for i, url in enumerate(urls):
    # Use regular expression to get file name(restaurants name) from url
    match = re.search(r'/biz/([\w-]+)(?=-san-francisco)', url)
    if match:
        # Add a order for future analysis
        filename = f"{i+1}-{match.group(1)}.html"
        file_path = os.path.join(folder_name, filename)
        response = requests.get(url)
        if response.status_code == 200:
            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(response.text)

### **Part 2  Parsing and Displaying Information from Saved HTML**

In [4]:

directory = 'restaurants_list'

restaurants_data = []

for filename in os.listdir(directory):
    if filename.endswith(".html"):
        match = re.search(r'^(\d+)-', filename)
        order_in_yelp = int(match.group(1))
        filepath = os.path.join(directory, filename)

        with open(filepath, 'r', encoding='utf-8') as file:
            html = file.read()
            soup = BeautifulSoup(html, 'html.parser')

            restaurant_info = {'Order in Yelp': order_in_yelp}

            # Title
            name = soup.select_one("h1.css-hnttcw")
            restaurant_info['Restaurant Name'] = name.get_text(strip=True) if name else 'N/A'

            # Rating
            rating = soup.select_one("span.css-1fdy0l5")
            restaurant_info['Rating'] = rating.get_text(strip=True) if rating else 'N/A'

            # Number of reviews
            number_of_reviews = soup.select_one("span.css-1x9ee72")
            restaurant_info['Number of Reviews'] = number_of_reviews.get_text(strip=True) if number_of_reviews else 'N/A'

            # Price level
            price_level = soup.select_one("span.css-14r9eb")
            restaurant_info['Price Level'] = price_level.get_text(strip=True) if price_level else 'N/A'

            # Cuisine
            cuisine = soup.select("a.css-19v1rkv")
            restaurant_info['Main Cuisine'] = cuisine[1].get_text(strip=True) if cuisine else 'N/A'

            # Free WIFI
            restaurant_info['Free Wi-Fi'] = 1 if soup.find(lambda tag: tag.name == "script" and 'Free Wi-Fi' in tag.text) else 0

            # Vegetarian
            restaurant_info['Vegetarian'] = 1 if soup.find(lambda tag: tag.name == "script" and 
                                                ('Many Vegetarian Options' in tag.text or 'Vegan Options' in tag.text)) else 0
            
            # Street Parking
            restaurant_info['Street Parking'] = 1 if soup.find(lambda tag: tag.name == "script" and 'Street Parking' in tag.text) else 0

            # Takes Reservations
            restaurant_info['Takes Reservations'] = 1 if soup.find(lambda tag: tag.name == "script" and 'Takes Reservations' in tag.text) else 0

            # Dogs Allowed
            restaurant_info['Dogs Allowed'] = 1 if soup.find(lambda tag: tag.name == "script" and 'Dogs Allowed' in tag.text) else 0

            # Good for Groups
            restaurant_info['Good for Groups'] = 1 if soup.find(lambda tag: tag.name == "script" and 'Good for Groups' in tag.text) else 0

            # Add the restaurant's information to the list
            restaurants_data.append(restaurant_info)

df_restaurants = pd.DataFrame(restaurants_data)

df_restaurants = df_restaurants.sort_values(by='Order in Yelp').reset_index(drop=True)
print(df_restaurants)

df_restaurants.to_csv('yelp_data.csv', index=False, encoding='utf-8-sig')

    Order in Yelp             Restaurant Name Rating Number of Reviews  \
0               1                     Bottega    4.3   (1,050 reviews)   
1               2                    The Snug    4.0     (439 reviews)   
2               3  Elena's Mexican Restaurant    4.1     (118 reviews)   
3               4             Kothai Republic    4.6     (269 reviews)   
4               5               Blind Butcher    4.0     (342 reviews)   
..            ...                         ...    ...               ...   
93             96       State Bird Provisions    4.2   (3,249 reviews)   
94             97                         N/A    N/A               N/A   
95             98                Cafe Okawari    4.5     (386 reviews)   
96             99                       Copra    4.0     (392 reviews)   
97            100                   Hook Fish    4.3   (1,143 reviews)   

   Price Level        Main Cuisine  Free Wi-Fi  Vegetarian  Street Parking  \
0           $$             Italia

In [5]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Order in Yelp       98 non-null     int64 
 1   Restaurant Name     98 non-null     object
 2   Rating              98 non-null     object
 3   Number of Reviews   98 non-null     object
 4   Price Level         98 non-null     object
 5   Main Cuisine        98 non-null     object
 6   Free Wi-Fi          98 non-null     int64 
 7   Vegetarian          98 non-null     int64 
 8   Street Parking      98 non-null     int64 
 9   Takes Reservations  98 non-null     int64 
 10  Dogs Allowed        98 non-null     int64 
 11  Good for Groups     98 non-null     int64 
dtypes: int64(7), object(5)
memory usage: 9.3+ KB


### **Part 3 Store data into a database**

Store data into a database

In [ ]:
warnings.filterwarnings("ignore")

SQL_DB = "db_422_final"

def create_sql_table(TABLE_NAME, TABLE_DEF):
    try:
        #connect to server
        conn = mysql.connector.connect(host='localhost',
                                       user='root',
                                       password='')
        cursor = conn.cursor()
        
        query = "CREATE DATABASE IF NOT EXISTS " + SQL_DB
        print(query)
        cursor.execute(query);
        
        query = "CREATE TABLE IF NOT EXISTS " + SQL_DB + "." + TABLE_NAME + " " + TABLE_DEF + ";";
        print(query)
        cursor.execute(query);
        cursor.close()
        conn.close()
        return

    except IOError as e:
        print(e)
    
conn = mysql.connector.connect(host='localhost',
                                user='root',
                                password='pmy0614..')
cur = conn.cursor()
drop_db = "DROP DATABASE IF EXISTS`db_422_final"
cur.execute(drop_db)

TABLE_NAME = "Restaurant_Info"
TABLE_DEF = "(" + \
            "Order_in_Yelp INT NOT NULL" + \
            ", Restaurant_Name VARCHAR(255) NOT NULL" + \
            ", Rating DECIMAL(2,1) NOT NULL" + \
            ", Number_of_Reviews INT NOT NULL" + \
            ", Price_Level VARCHAR(5) NOT NULL" + \
            ", Main_Cuisine VARCHAR(255) NOT NULL" + \
            ", Free_Wi_Fi TINYINT(1) NOT NULL" + \
            ", Vegetarian TINYINT(1) NOT NULL" + \
            ", Street_Parking TINYINT(1) NOT NULL" + \
            ", Takes_Reservations TINYINT(1) NOT NULL" + \
            ", Dogs_Allowed TINYINT(1) NOT NULL" + \
            ", Good_for_Groups TINYINT(1) NOT NULL" + \
            ", PRIMARY KEY (Order_in_Yelp, Restaurant_Name)" + \
            ")"


data_list = []
for index, row in df_restaurants.iterrows():
    data_tuple = (
        row['Order in Yelp'], 
        row['Restaurant Name'], 
        row['Rating'], 
        row['Number of Reviews'], 
        row['Price Level'], 
        row['Main Cuisine'], 
        row['Free Wi-Fi'], 
        row['Vegetarian'], 
        row['Street Parking'], 
        row['Takes Reservations'], 
        row['Dogs Allowed'], 
        row['Good for Groups']
    )
    data_list.append(data_tuple)


conn = mysql.connector.connect(host='localhost', database='db_422_final', user='root', password='')
cur = conn.cursor()


for data in data_list:
    insert_query = '''
        INSERT INTO Restaurant_Info (
            Order_in_Yelp, Restaurant_Name, Rating, Number_of_Reviews, Price_Level,
            Main_Cuisine, Free_Wi_Fi, Vegetarian, Street_Parking, Takes_Reservations,
            Dogs_Allowed, Good_for_Groups
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    cur.execute(insert_query, data)

conn.commit()

#### Add index

In [ ]:
index_queries = [
    "CREATE INDEX idx_login ON Github_info(Order_in_Yelp);",
    "CREATE INDEX idx_location ON Github_info(Restaurant_Name);"
]

for query in index_queries:
    cur.execute(query)

conn.commit()

cur.close()
conn.close()  